# Parsing through Database Tables

## Dependencies / Import Data

In [98]:
project_table['PA Permit Applied'].value_counts()

2019-08-12    66
2019-07-29    38
2019-08-01    33
2019-07-19    31
2019-07-11    31
2019-08-02    31
2019-07-08    30
2019-08-15    28
2019-07-30    28
2019-08-07    25
2019-07-24    24
2019-07-23    24
2019-08-05    24
2019-07-22    23
2019-07-25    22
2019-08-16    21
2019-08-09    21
2019-07-18    20
2019-06-19    20
2019-07-31    20
2019-06-21    18
2019-06-24    18
2019-07-15    17
2019-06-06    17
2019-06-27    16
2019-07-16    16
2019-06-07    16
2019-06-13    14
2019-07-03    14
2019-06-26    14
              ..
2019-05-09     4
2019-05-08     3
2019-05-31     3
2019-05-06     3
2019-06-14     3
2019-05-22     2
2019-05-23     2
2019-05-27     2
2019-05-15     2
2019-05-10     2
2019-05-21     2
2019-06-28     2
2019-04-16     2
2019-08-06     2
2019-04-25     1
2019-04-23     1
2019-04-15     1
2019-02-19     1
2019-05-28     1
2019-04-11     1
2019-04-26     1
2019-04-29     1
2019-07-02     1
2019-05-13     1
2019-08-23     1
2019-05-02     1
2019-04-05     1
2019-05-01    

In [89]:
# import data_prep
import numpy as np

import pandas as pd

############### PROJECT TABLE DATA ####################
project_table_data = "./data/database_tables/project_table.csv"
project_table = pd.read_csv(
    project_table_data, 
    dtype={'Claim #': str,'Job #': str,'Branch':str,'Claim Status':str},
    parse_dates=[
        'Rep Agreement Signed', 'Rep Claim Collected','FTA Scope Completed',
        'FTA Scope Rejected', 'BC Estimate Completed','OB Scope Completed',
        'Sup Job Submitted', 'BC Approved for Production', 'OB Order Built',
        'GM Order Processed', 'PA Permit Applied', 'PA Permit Processed',
        'PA OA Processed', 'PA OA Invoiced', 'PA Notify of Delivery',
        'PA Notify of Start', 'Delivery Date', 'Roof Start',
        'Roof End', 'GM Approved for Inspection', 'GM Change Order Date',
        'GM Labor Adjustment Date', 'RA Inspection Requested', 'RA Inspection Processed', 
        'Rep COC Collected', 'SA Job Docs Uploaded', 'BC Project Invoiced','BC Project Closed'])

############### PROJECT INFORMATION TABLE DATA ###############
project_info_data = "./data/database_tables/project_info_table.csv"

info_table = pd.read_csv(project_info_data, dtype={
    'Claim #':str, 'Job #':str, 'Branch':str, 'City':str, 'Building Department':str,
    'Permit Req?':str, 'Supplier Name':str, 'Crew':str, 'Insurance Company':str,
    'Multi-rejected':str, 'Scope Rejections':str, 'Change Orders':str,'Labor Adjustments':str})

# due to manual 'city' field, city is incorrectly spelled or off on capitalization
info_table['City'] = info_table['City'].str.upper()

#################### WORKFLOW TABLE DATA ####################
workflow_table_data = "./data/database_tables/workflow_table.csv"
workflow_table = pd.read_csv(workflow_table_data)

######################### EAGLEVIEW DATA #########################
eagleview_table_data = "./data/raw_datasets/[TVA] EagleView Analysis.csv"
eagleview_table = pd.read_csv(eagleview_table_data)

# converting the squares measurement to roofing SQs
eagleview_table['Square Feet'] = eagleview_table['Square Feet'] / 100

# removing all duplcates
eagleview_table = eagleview_table.drop_duplicates(subset='Claim #', keep='first')


# using a dict to point the 'selected_branch' function to the correct column
pipeline_dict = {  
    'Rep Claims': {'start':'Rep Agreement Signed', 'end':'Rep Claim Collected', 'other':'none',
                   'project_table':['Claim #', 'Job #', 'Branch', 'Claim Status', 'Rep Agreement Signed', 'Rep Claim Collected'],
                   'workflow_table':['Claim #','Rep Collecting Claim'],
                   'info_table':['Claim #', 'City'],
                   'eagleview_table':['Claim #', 'Square Feet'],
                   'clean_table':['Claim #','Job #','City','Rep Agreement Signed','Rep Claim Collected', 'Rep Collecting Claim','Square Feet','Branch','Claim Status',]
                  },
    'FTA Scopes': {'start':'Rep Claim Collected', 'end':'FTA Scope Completed', 'other':'none',
                   'project_table':['Claim #','Job #','Branch','Claim Status','Rep Claim Collected','FTA Scope Completed','FTA Scope Rejected',],
                   'workflow_table':['Claim #','FTA Completing Scope','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Scope Rejections',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':['Claim #', 'Job #','City','Rep Claim Collected','FTA Scope Completed','FTA Scope Rejected', 'FTA Completing Scope','Days in Pipeline','Scope Rejections','Building Department','Square Feet','Branch','Claim Status',],
                  }, 
    'BC Estimates': {'start':'FTA Scope Completed', 'end':'BC Estimate Completed', 'other':'none',
                   'project_table':['Claim #','Job #','Branch','Claim Status','FTA Scope Completed','BC Estimate Completed',],
                   'workflow_table':['Claim #','BC Completing Estimate','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':['Claim #', 'Job #', 'City', 'FTA Scope Completed','BC Estimate Completed', 'BC Completing Estimate', 'Days in Pipeline','Building Department', 'Square Feet','Branch', 'Claim Status',],
                    },
    'OB Scopes': {'start':'BC Estimate Completed', 'end':'OB Scope Completed', 'other':'none',
                   'project_table':['Claim #','Job #','Branch','Claim Status','BC Estimate Completed','OB Scope Completed',],
                   'workflow_table':['Claim #','OB Completing Scope','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Scope Rejections',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':['Claim #', 'Job #', 'City','BC Estimate Completed','OB Scope Completed','OB Completing Scope', 'Days in Pipeline', 'Scope Rejections','Building Department','Square Feet','Branch', 'Claim Status',],
                    },
    'Sup Submittals': {'start':'OB Scope Completed', 'end':'Sup Job Submitted', 'other':'Job #',
                   'project_table':['Claim #','Job #','Branch','Claim Status','OB Scope Completed','Sup Job Submitted',],
                   'workflow_table':['Claim #','Sup Submitting Job','Days in Pipeline',],
                   'info_table':['Claim #','City','Insurance Company',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':['Claim #', 'Job #','City','OB Scope Completed','Sup Job Submitted', 'Sup Submitting Job','Days in Pipeline',  'Insurance Company', 'Square Feet','Branch', 'Claim Status',],
                    },
    'BC Approvals': {'start':'Sup Job Submitted', 'end':'BC Approved for Production','other':'none',
                   'project_table':['Claim #','Job #','Branch','Claim Status','Sup Job Submitted','BC Approved for Production',],
                   'workflow_table':['Claim #','BC Approving Job','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Insurance Company',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':['Claim #', 'Job #','City','Sup Job Submitted','BC Approved for Production','BC Approving Job', 'Days in Pipeline','Insurance Company', 'Building Department','Square Feet','Branch', 'Claim Status',],
                    },    
    'OB Orders Built': {'start':'BC Approved for Production', 'end':'OB Order Built','other':'none',
                   'project_table':['Claim #','Job #','Branch','Claim Status','BC Approved for Production','OB Order Built',],
                   'workflow_table':['Claim #','OB Building Order','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Scope Rejections','Change Orders',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':['Claim #', 'Job #','City','BC Approved for Production', 'OB Order Built','OB Building Order','Days in Pipeline','Scope Rejections','Change Orders', 'Building Department','Square Feet','Branch', 'Claim Status',],
                    },
    'GM Orders Processed': {'start':'OB Order Built', 'end':'GM Order Processed','other':'none',
                   'project_table':['Claim #','Job #','Branch','Claim Status','OB Order Built','GM Order Processed',],
                   'workflow_table':['Claim #','GM Processing Order','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Permit Req?','Change Orders','Labor Adjustments',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':['Claim #', 'Job #', 'OB Order Built','GM Order Processed', 'GM Processing Order', 'Days in Pipeline', 'City','Change Orders','Labor Adjustments','Building Department','Permit Req?','Square Feet','Branch', 'Claim Status',],
                    },
    'PA Permits Applied': {'start':'GM Order Processed', 'end':'PA Permit Applied','other':'Permit Req?',
                   'project_table':['Claim #','Job #','Branch','GM Order Processed','PA Permit Applied',],
                   'workflow_table':['Claim #','PA Permit Applied','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Permit Req?',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'PA Permits Processed':{'start':'PA Permit Applied', 'end':'PA Permit Processed','other':'Permit Req?',
                   'project_table':['Claim #','Job #','Branch','PA Permit Applied','PA Permit Processed',],
                   'workflow_table':['Claim #','PA Permit Processed','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Permit Req?',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'PA OA Processed':{'start':'GM Order Processed', 'end':'PA OA Processed','other':'none',
                   'project_table':['Claim #','Job #','Branch','GM Order Processed','PA OA Processed',],
                   'workflow_table':['Claim #','PA Processing OA','Days in Pipeline',],
                   'info_table':['Claim #','City','Supplier Name','Change Orders',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'PA OA Invoiced':{'start':'PA OA Processed', 'end':'PA OA Invoiced','other':'none',
                   'project_table':['Claim #','Job #','Branch','PA OA Processed','PA OA Invoiced',],
                   'workflow_table':['Claim #','PA Invoicing OA','Days in Pipeline',],
                   'info_table':['Claim #','City','Supplier Name','Change Orders',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'PA Notify of Delivery':{'start':'Delivery Date', 'end':'PA Notify of Delivery','other':'none',
                   'project_table':['Claim #','Job #','Branch','PA Notify of Delivery','Delivery Date',],
                   'workflow_table':['Claim #','PA Notified of Delivery','Days in Pipeline',],
                   'info_table':['Claim #','City','Supplier Name','Change Orders',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'PA Notify of Start':{'start':'Roof Start', 'end':'PA Notify of Start','other':'none',
                   'project_table':['Claim #','Job #','Branch','PA Notify of Start','Roof Start',],
                   'workflow_table':['Claim #','PA Notified of Start','Days in Pipeline',],
                   'info_table':['Claim #','City','Crew','Labor Adjustments',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    }, 
    'GM Approved for Inspection':{'start':'Roof End', 'end':'GM Approved for Inspection','other':'Permit Req?',
                   'project_table':['Claim #','Job #','Branch','Roof End','GM Approved for Inspection',],
                   'workflow_table':['Claim #','GM Approving for Inspection','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Scope Rejections','Change Orders','Labor Adjustments',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'RA Inspection Requested':{'start':'GM Approved for Inspection', 'end':'RA Inspection Requested','other':'Permit Req?',
                   'project_table':['Claim #','Job #','Branch','GM Approved for Inspection','RA Inspection Requested',],
                   'workflow_table':['Claim #','RA Requesting Inspection','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Permit Req?',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'RA Inspection Processed':{'start':'RA Inspection Requested', 'end':'RA Inspection Processed','other':'Permit Req?',
                   'project_table':['Claim #','Job #','Branch','RA Inspection Requested','RA Inspection Processed',],
                   'workflow_table':['Claim #','RA Inspection Processed','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Permit Req?',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'Rep COC Collected':{'start':'Roof End', 'end':'Rep COC Collected','other':'none',
                   'project_table':['Claim #','Job #','Branch','Roof End','Rep COC Collected',],
                   'workflow_table':['Claim #','Rep Collecting COC','Days in Pipeline',],
                   'info_table':['Claim #','City','Insurance Company','Change Orders','Labor Adjustments',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'SA Job Docs Uploaded':{'start':'Rep COC Collected', 'end':'SA Job Docs Uploaded','other':'none',
                   'project_table':['Claim #','Job #','Branch','Rep COC Collected','SA Job Docs Uploaded',],
                   'workflow_table':['Claim #','SA Uploading Docs','Days in Pipeline',],
                   'info_table':['Claim #','City','Insurance Company','Change Orders','Labor Adjustments',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'BC Project Invoiced':{'start':'SA Job Docs Uploaded', 'end':'BC Project Invoiced','other':'none',
                   'project_table':['Claim #','Job #','Branch','SA Job Docs Uploaded','BC Project Invoiced',],
                   'workflow_table':['Claim #','BC Invoicing Project','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Insurance Company',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
    'BC Project Closed':{'start':'BC Project Invoiced', 'end':'BC Project Closed','other':'none',
                   'project_table':['Claim #','Job #','Branch','BC Project Closed',],
                   'workflow_table':['Claim #','BC Closed Project','Days in Pipeline',],
                   'info_table':['Claim #','City','Building Department','Insurance Company',],
                   'eagleview_table':['Claim #','Square Feet',],
                   'clean_table':[],
                    },
}


## Creating Workflow Class

In [99]:
class Branch_Workflow:
    
    def __init__(self, branch, pipeline):
        self.branch = branch
        self.pipeline = pipeline

    
    def create_teammate_workflow(self):
        
        # assigning the dict to a variable to get dict values
        pipeline_info = pipeline_dict.get(self.pipeline)
        
        # assigning variables to help define the pipeline chosen
        start = pipeline_info['start']
        end = pipeline_info['end']
        other = pipeline_info['other']
        project_list = pipeline_info['project_table']
        workflow_list = pipeline_info['workflow_table']
        info_list = pipeline_info['info_table']
        eagleview_list = pipeline_info['eagleview_table']
        master_list = pipeline_info['clean_table']
    
        # prodviding only revelant information from project table
        parsed_project_table = project_table[project_list]

        # providing only revelant information from workflow table
        parsed_workflow_table = workflow_table[workflow_list]

        # providing only revelant information from info table
        parsed_info_table = info_table[info_list]

        # providing only revelant information from eagleview table
        parsed_eagleview_table = eagleview_table[eagleview_list]

        # merging all parsed tables together
        merged_table = parsed_project_table.merge(
            parsed_workflow_table, on='Claim #', how='left').merge(
            parsed_info_table, on='Claim #', how='left').merge(
            parsed_eagleview_table, on='Claim #', how='left')
    
#         merged_table = merged_table[master_list]

        ###############
        # filtering by specific branch or ALL
        if (self.branch == 'All') | (self.branch == 'ALL'):
            branch_data = merged_table
        else:
            branch_data = merged_table.loc[(merged_table['Branch'] == self.branch), :]
        
        ###############
        # filtering the workflows without an 'other' condition
        if other == 'none':
            
            workflow_data = branch_data.loc[(
                branch_data[start].isnull() == False )&(
                branch_data[end].isnull() == False),:]
            
        # filtering the workflows dependent on permit requirements
        elif other == 'Permit Req?':
            
            workflow_data = branch_data.loc[(
                branch_data[start].isnull() == False)&(
                branch_data[end].isnull() == False)&(
                branch_data[other] == 'Y'),:]
            
        else:
            # pipelines with a specific 'other' value
            workflow_data = branch_data.loc[(
                branch_data[start].isnull() == False )&(
                branch_data[end].isnull() == False)&(
                branch_data[other].isnull() == True),:]
        
        ###############
        # reproducing the user input
        print(f'Branch being Analyzed: {self.branch}')
        print(f'Pipeline Chosen: {self.pipeline}')
        print('*'*40)
        
        print(f'Workflow Columns: {len(workflow_data.columns)}')
        print(workflow_data.columns)
        print('*'*40)
        
        print(f'Sorted Columns: {len(master_list)}')
        print(master_list)
        print('*'*40)
        
        return workflow_data
    
    

## User Pipeline Analysis

### Pipeline Analysis

In [24]:
def run_pipeline_analysis():
    
    # providing the branch options
    
    print('''
    #########################################################################################

                                        POSSIBLE BRANCHES

                                ALL | DEN | COS | FCO | KCI | OMA

    #########################################################################################
    ''')
    
    # storing the 'branch' response as a variable
    branch_response = input('Choose your Branch: ')
    
    # providing the pipeline options
    
    print('''
    
    #################################################################
    
                            POSSIBLE PIPELINES

    Rep Claims | FTA Scopes| BC Estimates | OB Scopes | Sup Submittals
    
    BC Approvals | OB Orders Built | GM Orders Processed | PA Permits Applied
    
    PA Permits Processed | PA OA Processed | PA OA Invoiced | PA Notify of Delivery
    
    PA Notify of Start | GM Approved for Inspection | RA Inspection Requested
    
    RA Inspection Processed | Rep COC Collected | SA Job Docs Uploaded 
    
    BC Project Invoiced | BC Project Closed
    
    ######################################################################
    
    ''')
    
    
    # storing the 'pipeline' response as a variable
    pipeline_response = input("Generate which Pipeline?: ")
    
    # creates 
    analysis = Branch_Pipeline(branch_response, pipeline_response)
    
    return analysis.create_teammate_pipeline()

### Workflow Analysis

In [95]:
def run_workflow_analysis():
    
    # providing the branch options
    
    print('''
    #########################################################################################

                                        POSSIBLE BRANCHES

                                ALL | DEN | COS | FCO | KCI | OMA

    #########################################################################################
    ''')
    
    # storing the 'branch' response as a variable
    branch_response = input('Choose your Branch: ')
    
    # providing the pipeline options
    
    print('''
    
    #################################################################
    
                            POSSIBLE PIPELINES

    Rep Claims | FTA Scopes| BC Estimates | OB Scopes | Sup Submittals
    
    BC Approvals | OB Orders Built | GM Orders Processed | PA Permits Applied
    
    PA Permits Processed | PA OA Processed | PA OA Invoiced | PA Notify of Delivery
    
    PA Notify of Start | GM Approved for Inspection | RA Inspection Requested
    
    RA Inspection Processed | Rep COC Collected | SA Job Docs Uploaded 
    
    BC Project Invoiced | BC Project Closed
    
    ######################################################################
    
    ''')
    
    
    # storing the 'pipeline' response as a variable
    workflow_response = input("Generate which Workflow?: ")
    
    # creates a 'workflow' Class instance and then passes the responses through it
    workflow_analysis = Branch_Workflow(branch_response, workflow_response)
    
    return workflow_analysis.create_teammate_workflow()

In [96]:
tester = run_workflow_analysis()

tester.head()


    #########################################################################################

                                        POSSIBLE BRANCHES

                                ALL | DEN | COS | FCO | KCI | OMA

    #########################################################################################
    
Choose your Branch: DEN

    
    #################################################################
    
                            POSSIBLE PIPELINES

    Rep Claims | FTA Scopes| BC Estimates | OB Scopes | Sup Submittals
    
    BC Approvals | OB Orders Built | GM Orders Processed | PA Permits Applied
    
    PA Permits Processed | PA OA Processed | PA OA Invoiced | PA Notify of Delivery
    
    PA Notify of Start | GM Approved for Inspection | RA Inspection Requested
    
    RA Inspection Processed | Rep COC Collected | SA Job Docs Uploaded 
    
    BC Project Invoiced | BC Project Closed
    
    #################################################################

KeyError: 'PA Permit Applied'